# Fine-tunning a model

In this example notebook we'll fine tune a [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2) with a [poquad dataset](https://huggingface.co/datasets/clarin-pl/poquad).

## Downloading poquad dataset

In [3]:
from datasets import load_dataset

poquad = load_dataset("clarin-pl/poquad")

## Downloading model

In [5]:
from transformers import AutoTokenizer

model_name = 'deepset/roberta-base-squad2'

tokenizer = AutoTokenizer.from_pretrained(model_name)

## Adding preprocessing function
It's from an hugging-face example. More information can be found here - https://huggingface.co/docs/transformers/tasks/question_answering

In [6]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

## Tokenizing the dateset

In [10]:
tokenized_poquad = poquad.map(preprocess_function, batched=True, remove_columns=poquad["train"].column_names)

Map:   0%|          | 0/5764 [00:00<?, ? examples/s]

## Fine-tuning model

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="output/roberta-base-squad2-pl",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poquad["train"],
    eval_dataset=tokenized_poquad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

  0%|          | 0/8661 [00:00<?, ?it/s]

{'loss': 2.366, 'grad_norm': 25.46889305114746, 'learning_rate': 1.884539891467498e-05, 'epoch': 0.17}
{'loss': 2.019, 'grad_norm': 31.06574249267578, 'learning_rate': 1.769079782934996e-05, 'epoch': 0.35}
{'loss': 1.8536, 'grad_norm': 25.971675872802734, 'learning_rate': 1.653619674402494e-05, 'epoch': 0.52}
{'loss': 1.7774, 'grad_norm': 28.223947525024414, 'learning_rate': 1.538159565869992e-05, 'epoch': 0.69}
{'loss': 1.7428, 'grad_norm': 25.615482330322266, 'learning_rate': 1.42269945733749e-05, 'epoch': 0.87}


  0%|          | 0/361 [00:00<?, ?it/s]

{'eval_loss': 1.5653996467590332, 'eval_runtime': 91.9805, 'eval_samples_per_second': 62.665, 'eval_steps_per_second': 3.925, 'epoch': 1.0}
{'loss': 1.6227, 'grad_norm': 44.27663803100586, 'learning_rate': 1.3072393488049879e-05, 'epoch': 1.04}
{'loss': 1.4481, 'grad_norm': 27.834014892578125, 'learning_rate': 1.1917792402724858e-05, 'epoch': 1.21}
{'loss': 1.4219, 'grad_norm': 34.01536560058594, 'learning_rate': 1.076319131739984e-05, 'epoch': 1.39}
{'loss': 1.431, 'grad_norm': 24.65727996826172, 'learning_rate': 9.60859023207482e-06, 'epoch': 1.56}
{'loss': 1.4034, 'grad_norm': 43.74335861206055, 'learning_rate': 8.453989146749799e-06, 'epoch': 1.73}
{'loss': 1.3625, 'grad_norm': 39.83943176269531, 'learning_rate': 7.299388061424778e-06, 'epoch': 1.91}


  0%|          | 0/361 [00:00<?, ?it/s]

{'eval_loss': 1.4093455076217651, 'eval_runtime': 89.9364, 'eval_samples_per_second': 64.09, 'eval_steps_per_second': 4.014, 'epoch': 2.0}
{'loss': 1.282, 'grad_norm': 24.49555015563965, 'learning_rate': 6.144786976099758e-06, 'epoch': 2.08}
{'loss': 1.187, 'grad_norm': 29.855417251586914, 'learning_rate': 4.990185890774737e-06, 'epoch': 2.25}
{'loss': 1.1464, 'grad_norm': 21.781904220581055, 'learning_rate': 3.835584805449718e-06, 'epoch': 2.42}
{'loss': 1.1625, 'grad_norm': 25.76011848449707, 'learning_rate': 2.680983720124697e-06, 'epoch': 2.6}
{'loss': 1.1572, 'grad_norm': 40.99302673339844, 'learning_rate': 1.5263826347996768e-06, 'epoch': 2.77}
{'loss': 1.147, 'grad_norm': 25.29827880859375, 'learning_rate': 3.7178154947465653e-07, 'epoch': 2.94}


  0%|          | 0/361 [00:00<?, ?it/s]

{'eval_loss': 1.4216176271438599, 'eval_runtime': 90.2408, 'eval_samples_per_second': 63.874, 'eval_steps_per_second': 4.0, 'epoch': 3.0}
{'train_runtime': 8207.5606, 'train_samples_per_second': 16.882, 'train_steps_per_second': 1.055, 'train_loss': 1.4954243963770333, 'epoch': 3.0}


TrainOutput(global_step=8661, training_loss=1.4954243963770333, metrics={'train_runtime': 8207.5606, 'train_samples_per_second': 16.882, 'train_steps_per_second': 1.055, 'train_loss': 1.4954243963770333, 'epoch': 3.0})

## Testing new model

In [17]:
from transformers import pipeline

model = AutoModelForQuestionAnswering.from_pretrained("output/roberta-base-squad2-pl/checkpoint-8500")
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

context = 'Maria Skłodowska-Curie była polską i naturalizowaną francuską fizyczką i chemiczką, która prowadziła pionierskie badania nad promieniotwórczością. Była pierwszą kobietą, która zdobyła Nagrodę Nobla, pierwszą osobą i jedyną, która zdobyła Nagrody Nobla w dwóch różnych dziedzinach nauki, i była częścią rodziny Curie, która zdobyła pięć Nagród Nobla.'
question = 'W jakiej dziedzinie Maria Curie prowadziła pionierskie badania?'

result = nlp(question=question, context=context)

print(result)

{'score': 0.5960702300071716, 'start': 125, 'end': 145, 'answer': 'promieniotwórczością'}


We can see a huge improvement in score. From 0.02 on base model to 0.59 on this fine-tuned model. However the answer it not correct, as it would need to be correctly conjugated to `promieniotwórczości` and not `promieniotwórczością`.